# Libraries

In [ ]:
# !pip install Unidecode
# !pip install datashader
# !pip install geopandas
# !pip install haversine

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import tqdm
from unidecode import unidecode
from haversine import haversine, Unit

In [ ]:
import sys
import os

# File path to the src directory for both linux and windows
# workaround for the issue of relative imports in Jupyter notebooks to import modules from src without using the full path
src_path = os.path.abspath("src")
if src_path not in sys.path:
    sys.path.insert(0, src_path)

In [ ]:
# Rerun this cell after making changes to the utils module
from the_team.utils import etl, viz
import importlib
importlib.reload(etl)
importlib.reload(viz)

# Set custom plot style for consistency
viz.set_plot_style()

# Before Cleaning

In [ ]:
RAW_DIR = Path("data") / "01_raw"

In [ ]:
# Load datasets
customers = etl.load_csv(RAW_DIR / "olist_customers_dataset.csv")
geolocation = etl.load_csv(RAW_DIR / "olist_geolocation_dataset.csv")
items = etl.load_csv(RAW_DIR / "olist_order_items_dataset.csv")
payments = etl.load_csv(RAW_DIR / "olist_order_payments_dataset.csv")
reviews = etl.load_csv(RAW_DIR / "olist_order_reviews_dataset.csv")
orders = etl.load_csv(RAW_DIR / "olist_orders_dataset.csv")
products = etl.load_csv(RAW_DIR / "olist_products_dataset.csv")
sellers = etl.load_csv(RAW_DIR / "olist_sellers_dataset.csv")
translation = etl.load_csv(RAW_DIR / "product_category_name_translation.csv")

### Geolocation-related datasets [Jin Bin]
- customers.csv
- geolocation.csv
- sellers.csv

#### customers.csv

In [ ]:
customers.head()

In [ ]:
customers.info()

In [ ]:
# Check for duplicates
etl.null_duplicate_check(customers)

In [ ]:
# Data formatting
formatted_customers = etl.format_customers(customers)

In [ ]:
# Min seems low but its not a problem 
formatted_customers.describe()

In [ ]:
viz.plot_top_locations(formatted_customers, title_prefix="Customer")

1. Strong Regional Concentration.(SP) dominates customer count with over 40,000 customers — nearly half of the data. This suggests regional market dependence so if Olist wants to target repeat buyers, SP should be a priority.

2. Urban Centers Drive Volume. Cities like São Paulo, Rio de Janeiro, and Belo Horizonte are far ahead of others. (Urban hubs = higher density = possibly faster repeat behavior.) --> we could analyze if urban customers reorder more frequently due to better delivery coverage or seller availability after taking sellers.csv into account.

#### geolocation.csv

In [ ]:
geolocation.head()

The geolocation dataset contains multiple similar latitude and longitude entries for the same zip code prefix. To simplify the data and enable efficient merging with customer and seller datasets, we averaged the latitude and longitude for each unique zip code prefix. While this reduces geographic precision, it preserves regional location context needed for distance-based analysis in later stages.

In [ ]:
geolocation.info()

In [ ]:
#checks for duplicates
etl.null_duplicate_check(geolocation, verbose=False)

In [ ]:
geolocation.describe()

max lat/lng values are a little suspicious cause the borders of brazil are not that big
- lat range should be between 33.75116944 and 5.27438888
- lng range should be between -73.98283055 and -34.79314722

In [ ]:
# Defining Brazil's boundary conditions
valid_lat_range = (-33.75116944, 5.27438888)
valid_lng_range = (-73.98283055, -34.79314722)

# Prepare data
lat_outliers = geolocation[
    (geolocation["geolocation_lat"] < valid_lat_range[0]) |
    (geolocation["geolocation_lat"] > valid_lat_range[1])
]["geolocation_lat"]

lng_outliers = geolocation[
    (geolocation["geolocation_lng"] < valid_lng_range[0]) |
    (geolocation["geolocation_lng"] > valid_lng_range[1])
]["geolocation_lng"]

# Plot base boxplots
plt.figure(figsize=(10, 5))
sns.boxplot(data=geolocation[["geolocation_lat", "geolocation_lng"]], orient="h", palette="Set2")

# Overlay red dots for out-of-bound lat/lng
plt.scatter(lat_outliers, ["geolocation_lat"] * len(lat_outliers), color="red", label="Out-of-Range")
plt.scatter(lng_outliers, ["geolocation_lng"] * len(lng_outliers), color="red")

plt.title("Boxplot of Latitude and Longitude (Geolocation)")
plt.xlabel("Value")
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
# Removing the outlier Lat/Lng values 
# Filter only valid rows
geolocation = geolocation[
    (geolocation["geolocation_lat"].between(*valid_lat_range)) &
    (geolocation["geolocation_lng"].between(*valid_lng_range))
]

# Check if any outliers remain
print(f"Remaining rows: {len(geolocation)}")
geolocation.describe()

In [ ]:
formatted_geolocation = etl.format_geolocation(geolocation)
formatted_geolocation.head()

will be merged with customers.csv and sellers.csv to plot a map for the distribution of sellers and customers. Could also be used to calculate distance between the 2 groups.

#### sellers.csv

In [ ]:
sellers.head()

In [ ]:
sellers.info()

In [ ]:
#check for duplicates
etl.null_duplicate_check(sellers)

In [ ]:
formatted_sellers = etl.format_sellers(sellers)

In [ ]:
formatted_sellers.describe()

In [ ]:
viz.plot_top_locations(formatted_sellers, 
                   state_col="seller_state", 
                   city_col="seller_city", 
                   title_prefix="Seller")


Customer and seller geographic distribution shows strong overlap in urban regions such as São Paulo and Rio de Janeiro. This urban concentration, combined with higher seller density, could facilitate quicker deliveries and higher satisfaction—factors known to influence repeat buying behavior.

### Review-related datasets [Habib]
- review.csv
- products.csv
- translation.csv

#### Review dataset


In [ ]:
reviews.head()

In [ ]:
reviews.info()

In [ ]:
# .info shows missing entries
etl.null_duplicate_check(reviews)

In [ ]:
# Step 1) Since many users about 90% did not give their comments a title I will drop the title column as the data is not complete enough, the review date/timestamp are not relevant as 
# there is no time aspect in this situation so that will be dropped as well. The Review Id is also not required as its just used to index the reviews and this is not relevant.

reviews.drop(columns=["review_id", "review_comment_title","review_creation_date", "review_answer_timestamp"], inplace=True)

reviews.head()

# Dataset is now less bloated and only relevant columns remain.

In [ ]:
# Lets take a look at the spread of the review scores before any validation.

viz.plot_categorical_distribution(reviews, "review_score")

- We can see that there is a large bias towards 5 & 4 point reviews suggesting about 1 in 3 orders are satisfactory and leave a posititve impression on the end user.
- However there is a large amount of neutral reviews. To prevent non useful datapoints, this will be checked against the sentiment of the comments to verify all ratings are as intended.

#### Products dataset

In [ ]:
products.head()

In [ ]:
products.info()

In [ ]:
# .info shows small amount of missing entries
etl.null_duplicate_check(products)

In [ ]:
# 2) There is no possible way to feature engineer or substitute placeholder values for any of the columns so I will drop any df row with no data entries

products.dropna(inplace=True)

etl.null_duplicate_check(products)


In [ ]:
# 3) Convert float to int for certain columns

products = products.astype({
    "product_name_lenght": int,
    "product_description_lenght": int,
    "product_photos_qty": int
})

# Checking all dt conversions worked as intended
products.info()

#### Translation Dataset

In [ ]:
translation.head()
# From what I can see, this is a small dataset for end users to translate the portuguese labeled items.

In [ ]:
translation.info()
# There are no missing or duplicate values. Nor any further visualisations, so I will leave this dataset as is and may use it in the future for sentiment analysis of product categories.

### Order-related datasets [Min]
- items.csv
- payments.csv
- orders.csv

#### items.csv

In [ ]:
items.head()

In [ ]:
items.info()

In [ ]:
# df info says no Null but still; Null and duplicate check
etl.null_duplicate_check(items)

In [ ]:
# 1. shipping_limit_date has wrong data type.
items["shipping_limit_date"] = etl.to_datetime(items["shipping_limit_date"])
assert items["shipping_limit_date"].dtype == "datetime64[ns]", "shipping_limit_date should be datetime64[ns]"

# order_item_id is categircal, so object type is fine

# Some duplicates are expected after dropping order_item_id, since there can be multiple items in an order.
# Left untreated as these duplicates are identifiable and workable with the order_id.
etl.null_duplicate_check(items, verbose=False)

In [ ]:
# Check distribution of numeric columns
items.describe()

- A large gap between the 75th percentile (Q3) and the maximum in prices indicates that there are high-value outliers. 
- Olist dataset distributor mentioned (in Kaggle) that the data is from 2016 to 2018.

In [ ]:
# 2. Drop rows not within 2016-2018
items = items[items["shipping_limit_date"].dt.year.isin([2016, 2017, 2018])]

# Plot distribution of numeric columns
viz.plot_numeric_distribution(items.drop("order_item_id", axis=1))

- Both price & freight_value columns show strong right-skewed distributions with a large number of high-value outliers, which is expected as people mostly buy FMCGs (with low cost) from online, not high-end products.
- Right-skewed price means the item is expensive while right-skewed freight_value means either the customer lives far away from sellers or the item is physically huge; this can be cross-checked later with geolocation and product data.

In [ ]:
price_99p = items['price'].quantile(0.999)
print(f"99.9% of price is ${price_99p}.")
freight_99p = items['freight_value'].quantile(0.999)
print(f"99.9% of freight_value is ${freight_99p:.2f}.")

High values in both price and freight_value are important as they may suggest:
- People who bought expensive quality producuts are less likely to buy again. 
- People who have to pay high freight_value are less likely to buy again. 
But, capturing 99.9% of current customers should be representative enough. 

In [ ]:
# 3. Remove outliers
items['price'] = etl.cap_outliers(items['price'], min_cap=False, max_cap=.999)
items['freight_value'] = etl.cap_outliers(items['freight_value'], min_cap=False, max_cap=.999)

In [ ]:
# 4. Flag high values for the model to learn that a price/freight is unusually high (usual outliers) 
items['price_high'] = items['price'] > items['price'].quantile(0.75) * 1.5
items['freight_value_high'] = items['freight_value'] > items['freight_value'].quantile(0.75) * 1.5
items.freight_value_high.value_counts()

#### orders.csv

In [ ]:
orders.head()

In [ ]:
orders.info()

In [ ]:
# 1. Convert to datetime and clip to 2016-2018
for col in orders.columns[3:]:
    orders[col] = etl.clip_datetime(orders[col])
    assert orders[col].dtype == "datetime64[ns]", f"{col} should be datetime64[ns]"

- order_delivered_carrier_date (which marks when the seller handed the package to the carrier) is mainly for logistic purposes and does not influence repeat buyer behaviour as comparied to the duration between purchase and delivery.

In [ ]:
# 2. Drop order_delivered_carrier_date
orders.drop(columns=["order_delivered_carrier_date"], inplace=True)
assert "order_delivered_carrier_date" not in orders.columns, "order_delivered_carrier_date should be dropped"

# Null values
etl.null_duplicate_check(orders)

In [ ]:
# ~3% of the data shows that the order_delivered_customer_date is null
# meaning that the order was not delivered or have not been delivered yet to the customer

# Check if null dates are related to different order statuses
null_dates_orders = orders[orders.isnull().any(axis=1)].copy()
viz.plot_categorical_distribution(null_dates_orders, "order_status", "Count of null dates by order status")

- Since the goal is to identify potential repeat buyer, we will focus on 'paid' statuses; dropping others while trying to impute the null values.

In [ ]:
# 3. Drop rows with [canceled, created, invoiced, unavailable] order_status
orders = orders[~orders["order_status"].isin(["canceled", "created", "invoiced", "unavailable"])]

# Vislualize the duration between order_approved_at and order_delivered_customer_date
mean, median, mode = viz.plot_duration_distribution(
                        df=orders,
                        column_x='order_delivered_customer_date',
                        column_y='order_estimated_delivery_date',
                        title='Delivery Duration Distribution between Estimated and Actual Delivery Dates',
                    )

- The duration is right-skewed, and the mean is dragged towards right due to extreme outliers. 

In [ ]:
# 4. Fill null values in order_delivered_customer_date with the median duration
orders["order_delivered_customer_date"] = orders["order_delivered_customer_date"].fillna(orders["order_estimated_delivery_date"] - median) # type: ignore
assert orders["order_delivered_customer_date"].isnull().sum() == 0, "There should be no null values in order_delivered_customer_date"

# For null order_approved_at,
orders[orders.order_approved_at.isna()]['order_status'].value_counts()

- order_status shows delivered, but order_approved_at (payment time) and order_delivered_customer_date is null, meaning these are mix-mactched; incorrect data (maybe the customer used points to exchange instead of payment?)


In [ ]:
# 5. drop NULL in order_approved_at
orders.dropna(inplace=True)
assert orders.isnull().sum().sum() == 0, "There should be no null values in the dataset as of now."

#### payments.csv

In [ ]:
payments.head()

In [ ]:
payments.info()

In [ ]:
# They are all in correct data types.
etl.null_duplicate_check(payments)

In [ ]:
# it seems the school intentionally screw the data btw ^^
payments.order_id.nunique()

In [ ]:
# Which payment types are commonly used?
viz.plot_categorical_distribution(payments, "payment_type", "Count of Payment Types")

In [ ]:
# A glance at installments
viz.plot_numeric_distribution(payments.drop(columns=["order_id", "payment_sequential", "payment_type", "payment_value"]))

- There is no info on not_defined payment type and logically does not make sense; dropped.
- payment_installments is right-skewed, and that many times of installments are not common either; capped.
- payment_sequential is based on customer preference and is redundant; we can work with groupby later if necessary; dropped.

In [ ]:
# 1. drop not_defined payment types
payments = payments[payments["payment_type"] != "not_defined"]
assert "not_defined" not in payments["payment_type"].unique(), "not_defined payment type should be dropped"

# 2. cap installments to 99% quantile
payments["payment_installments"] = etl.cap_outliers(payments["payment_installments"], min_cap=False, max_cap=True)

# 3. drop payment_sequential as it is not relevant for the analysis; payment_value has been analyzed in the items dataset, so we can drop it here
payments.drop(columns=["payment_sequential"], inplace=True)
assert "payment_sequential" not in payments.columns, "payment_sequential should be dropped"

In [ ]:
df = payments.groupby("order_id")["payment_installments"].nunique()
multiple_installments = df[df > 2].index.values
payments[payments["order_id"].isin(multiple_installments[:1])]

- People are weird; for the same order, he chose to pay with a voucher, a credit card with 4 installments and another credit card with 6 installments. 
- Since this is also a part of Olist's flexible payment, these installments will just be added together to reduce complexity while keeping info.